In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
import PIL
import re
import utils

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used
plt.ion()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()

In [ ]:
# TODO: Dispay 5 images of keypoints working with humans
# TODO: Display 5 images of wolves with ground truth keypoints
# TODO: Display 5 images of wolves with ground truth bounding boxes

In [ ]:
# Implement pretrained keypoint_rcnn for keypoint detection on human picture
model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True)

model.eval()

image = cv2.imread('images/player2.jpg')
image_tensor = transforms.functional.to_tensor(image)

output = model([image_tensor])

for instance in output[0]["keypoints"]:
    for kp in instance:
        cv2.circle(image, (int(kp[0]), int(kp[1])), int(3*kp[2]), (127, 200, 127), 2, cv2.LINE_AA)
    
cv2.imshow("img", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Loading data using transfer learning tutorial from pytorch

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(256),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'Datasets/Synthetic_Wolf_1__2019_07_18/Images/1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=False, num_workers=0) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


In [69]:
#Load and process dataset for input to keypoint_rcnn model
class LoadQuadrupedDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.images = list(os.listdir(os.path.join(root)))

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        # [Images]
        index =  index + 1
        img_path = os.path.join(self.root, self.images[index])
        img = PIL.Image.open(img_path).convert("RGB")

        # [Targets]
        annotation_path = 'Datasets/Synthetic_Wolf_1__2019_07_18/Annotation/BonePositions.txt'
        file = open(annotation_path, "r")

        target = {}
        keypoints_data = []
        boxes_data = []
        
        for i, line in enumerate(file):
            # Skip the first iteation as it contains the joints name from the text file (BonePositions)
            if i == 0:
                continue
                
            # Get the keypoint data for the right frame represented by the index
            if i == index:
                individual_cordinates = line.split(' ')[0]
                individual_cordinates = re.split(r'\t+', individual_cordinates)
                
                for j, cordinate in enumerate(individual_cordinates):
                    # Skip even iterations as including it duplicates (x,y) cordinates entry
                    if j%2 == 0:
                        continue
                    # Skip the value in the first index of the list as this is the frame number    
                    if j is not 0:
                        keypoints_data.append([float(cordinate), float(individual_cordinates[j + 1]), float(1)])
                        if j is 1:
                            # Obtain the boxes of each samples, theese are: xmin, xmax, ymin, ymax
                            xmin = float(cordinate)
                            xmax = float(cordinate)
                            ymin = float(individual_cordinates[j + 1])
                            ymax = float(individual_cordinates[j + 1])
                            # boxes_data[i] = torch.FloatTensor([float(xmin), float(ymin), float(xmax), float(ymax)])
                            boxes_data.insert(0, np.array([float(xmin), float(ymin), float(xmax), float(ymax)]))
                        else:
                            xmin = xmin if (xmin < float(cordinate)) else float(cordinate)
                            xmax = xmax if (xmax > float(cordinate)) else float(cordinate)
                            ymin = ymin if (ymin < float(individual_cordinates[j + 1])) else float(individual_cordinates[j + 1])
                            ymax = ymax if (ymax > float(individual_cordinates[j + 1])) else float(individual_cordinates[j + 1])
                            #boxes_data[i] = torch.FloatTensor([float(xmin), float(ymin), float(xmax), float(ymax)])
                            boxes_data.insert(0, np.array([float(xmin), float(ymin), float(xmax), float(ymax)]))
            
            # Break out of loop once desired image and infomration are collected
            break
        
        
        target["labels"] = torch.ones((1,), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints_data, dtype=torch.float32)
        target["boxes"] = torch.as_tensor(boxes_data, dtype=torch.float32)

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        # Closing opened file
        file.close()
        
        # Delete unused variables
        del keypoints_data
        del boxes_data
        del img_path
        torch.cuda.empty_cache()
        
        return img, target

In [66]:
import transforms as T

def get_transform(train):
    transforms_container = []
    transforms_container.append(T.ToTensor())
    return T.Compose(transforms_container)

In [47]:
LoadQuadrupedDataset('Datasets/Synthetic_Wolf_1__2019_07_18/Images/1')[0]

(4,)
[ 98.66721 141.8787  131.5912  232.3395 ]


(<PIL.Image.Image image mode=RGB size=256x256 at 0x1EDCDE0C400>,
 {'labels': tensor([1]), 'keypoints': tensor([[111.2445, 169.4638,   1.0000],
          [110.2057, 157.4300,   1.0000],
          [109.3295, 148.9941,   1.0000],
          [110.3106, 143.4206,   1.0000],
          [116.3418, 141.8787,   1.0000],
          [115.8450, 154.1305,   1.0000],
          [122.6154, 157.9207,   1.0000],
          [129.8520, 177.2709,   1.0000],
          [120.1190, 181.8350,   1.0000],
          [112.5925, 187.8032,   1.0000],
          [103.7195, 154.1634,   1.0000],
          [ 98.8244, 159.7646,   1.0000],
          [ 98.6672, 180.0651,   1.0000],
          [109.4668, 196.5307,   1.0000],
          [110.9072, 196.9385,   1.0000],
          [120.1737, 183.9141,   1.0000],
          [121.5873, 190.9700,   1.0000],
          [116.7371, 218.4954,   1.0000],
          [114.4588, 232.2895,   1.0000],
          [105.0579, 185.5270,   1.0000],
          [102.0468, 194.1914,   1.0000],
          [105.29

In [ ]:
# Custom model traning method 1
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for images, target in dataLoader:
                #images = list(image.to(device) for image in images)
                #target = [{k: v.to(device) for k, v in t.items()} for t in target]
                
                images = inputs.to(device)
                targets = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images, target)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, targets)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == targets.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [70]:
# Training using inbuilt method
from engine import train_one_epoch, evaluate

dataset = LoadQuadrupedDataset('Datasets/Synthetic_Wolf_1__2019_07_18/Images/1', get_transform(train=True))
dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)

keypoint_model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True, progress=True, num_classes=2, num_keypoints=17, pretrained_backbone=True)
#out = nn.ConvTranspose2d(512, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
#keypoint_model.roi_heads.keypoint_predictor.kps_score_lowres = out

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    # Place model in a designated GPU from the inputs
    keypoint_model = nn.DataParallel(keypoint_model).cuda(1)
keypoint_model = keypoint_model.to(device)

criterion = nn.CrossEntropyLoss()
params = [p for p in keypoint_model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Deleting Variables
del dataset
torch.cuda.empty_cache()

In [ ]:
num_epochs = 10


# Custom training method #2
valid_loss_min = np.inf
epoch_resume = 0
for epoch in range (1, num_epochs+1):
    epoch += epoch_resume
    
    train_loss_per_batch = []
    val_loss_per_batch = []
    
    for images, targets in dataLoader:
        images = list(image.to(device) for image in images)
        targets= [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = loss_dict['loss_keypoint']

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss_per_batch.append(losses.item())

    # Validation
    with torch.no_grad():
        for images, targets in valid_loader:
            images = list(image.to(device) for image in images)
            targets= [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = keypoint_model(images, targets)
            losses = loss_dict['loss_keypoint']
            val_loss_per_batch.append(losses.item())

    avg_val_loss = np.mean(np.array(val_loss_per_batch))
    avg_train_loss = np.mean(np.array(train_loss_per_batch))
    print(f"Epoch [{epoch}]\t- Train Loss: {avg_train_loss}\t- Validation Loss: {avg_val_loss} ")

    if avg_val_loss <= valid_loss_min:
        print(f"\nValidation loss decreased ({valid_loss_min} --> {avg_val_loss}).  Saving model ...\n")
        #orch.save({
               #'epoch': epoch,
                #model_state_dict': keypoint_model.state_dict(),
               #'optimizer_state_dict': optimizer.state_dict(), 
              # 'best_loss': avg_val_loss
           #}, saved_model_path)

        valid_loss_min = avg_val_loss

In [71]:
num_epochs = 1

#model_ft = train_model(keypoint_model, criterion, optimizer, lr_scheduler, num_epochs=num_epochs)

for epoch in range(num_epochs):
    train_one_epoch(keypoint_model, optimizer, dataLoader, device, epoch, print_freq=10)
    lr_scheduler.step()
    torch.cuda.empty_cache()
    #evaluate(keypoint_model, dataLoader, device=device)


KeyboardInterrupt: 

In [ ]:
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except: pass